<a href="https://colab.research.google.com/github/YolandaNkalashe25/COS801/blob/main/COS_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision
import torch
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [2]:
import numpy as np
from torch import nn, optim
import torch.nn.functional as F

In [ ]:
Fashion_MNIST_DF=pd.read_csv('/content/drive/My Drive/Colab Notebooks/COS801/fashion-mnist_train.csv')

In [ ]:
Fashion_MNIST_DF.head()


In [ ]:
col=Fashion_MNIST_DF.columns

In [ ]:
Fashion_MNIST_DF=Fashion_MNIST_DF.replace('?',np.nan)

In [ ]:
Fashion_MNIST_DF=Fashion_MNIST_DF.dropna()

In [ ]:
import seaborn as sns
sns.countplot(Fashion_MNIST_DF.label)

In [ ]:
df_features=Fashion_MNIST_DF[col[1:]]
df_target=Fashion_MNIST_DF[col[0]]

In [ ]:
#standardize the value.
x=df_features.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_features= pd.DataFrame(x_scaled)


In [ ]:
df_features.columns=col[1:]

In [ ]:
df_features.head()

In [ ]:
# Split data into training and validation set.
X_train, x_val, Y_train, y_val=train_test_split(df_features,df_target,test_size=0.2,random_state=7)

In [ ]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
Y_train = torch.squeeze(torch.from_numpy(Y_train.to_numpy()).float())
x_val = torch.from_numpy(x_val.to_numpy()).float()
y_val = torch.squeeze(torch.from_numpy(y_val.to_numpy()).float())


In [ ]:
print(X_train.shape, Y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
input_features=784
output_size=10
hidden_size=0

In [ ]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_features, output_size)

  def forward(self, x):
    x = F.log_softmax(self.fc1(x),dim=1)
    return x

In [ ]:
model=Net()

In [ ]:
Y_train=Y_train.type(torch.LongTensor)
y_val=y_val.type(torch.LongTensor)

In [ ]:
#Optimizer to update the weights during backpropagation use stochatic gradient descent optimizer. We instantiate with our model parameters
optimizer = optim.SGD(model.parameters(),lr=0.01)

Loss_fn=nn.NLLLoss()


In [ ]:
ephoch_data=[]
ephoc=10001
batch_size=50

In [ ]:
import tensorflow as tf

In [ ]:
y_val.size()[0]

In [11]:
class FashionMnist(Dataset):

  def __init__(self):
    #data loading
    xy=np.loadtxt('/content/drive/My Drive/Colab Notebooks/COS801/fashion-mnist_train.csv',skiprows=1)
    self.x=torch.from_numpy(xy[:, 1:])
    sef.y=torch.from_numpy(xy[:,[0]])
    self.n_samples = xy.shape[0]

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.n_samples

dataset=FashionMnist()
first_data = dataset[0]
features, labels = first_data
print(features, labels)


NameError: ignored

In [ ]:
dataset=MNISTDataset

In [ ]:
#Start training for a 1000 ephocs.
#for every ephoc we zero out our gradients so that we have fresh gradients:
for ephoch in range(1,ephoc):
  for i in range(batch_size):
    

  optimizer.zero_grad()
  Ypred=model(X_train)

  #We then perform a backward pass.The model only has a function for a forward pass.

  loss=Loss_fn(Ypred, Y_train)
  loss.backward()  ## Calculate the gradients once loss has been calculated.

  #Once gradients are calculated we need to update our parameters:

  optimizer.step()

  ##For every ephoc we calculate the loss on the validation set to see how our data performs on the validation set:
  Ypred_validation=model(x_val)
  loss_validation=Loss_fn(Ypred_validation, y_val)
  # we do not call the backward function when testing on our validation set because we do not use this loss value to update the weights.

  _,pred=Ypred_validation.data.max(1) # Our prediction is the value with the highest probabilty

  #For every ephoch we print out the accuracy of the validation set and the loss.
  accuracy =pred.eq(y_val.data).sum().item()/y_val.size()[0]
  ephoch_data.append([ephoch,loss.data.item(),loss_validation.data.item(),accuracy])
  
  #for every 100 ephocs print out the data to screen

  if ephoch % 100==0:
    print('ephoc- %d (%d%%) train loss - %.2f validation loss - %.2f accuracy - %.4f'\
             % (ephoch, ephoch/150 * 10, loss.data.item(),loss_validation.data.item(),accuracy))


In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
Fashion_MNIST_DF=pd.read_csv('/content/drive/My Drive/Colab Notebooks/COS801/fashion-mnist_train.csv')

In [ ]:
Fashion_MNIST_DF.shape

(60000, 785)

In [ ]:
trainset = torch.utils.data.DataLoader(Fashion_MNIST_DF, batch_size=50, shuffle=True, num_workers=2)


In [ ]:
trainset.dataset.shape

(60000, 785)

In [ ]:
trainset.dataset['label']


0        2
1        9
2        6
3        0
4        3
        ..
59995    9
59996    1
59997    8
59998    8
59999    7
Name: label, Length: 60000, dtype: int64